In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import os
import fitz
import pytesseract
import io
import sys

from pprint import pprint
from PIL import Image
from pdf2image import convert_from_path
from platform import system
from difflib import get_close_matches

import util
import lines
import group
import label
import records
import extract
import date

In [12]:
test_files_dir = ".././test_files/"
test_dfs_dir = test_files_dir + "tesseract_data_frames/"
test_output_dir = test_files_dir + "extracted_indexes/"
test_files = util.list_files(test_files_dir, suffix=".pdf")
test_dfs = util.list_files(test_dfs_dir)
file = test_dfs[-3]

indexes_path = "/home/jeusa/Files/Documents/Indexex/ILO_yearly_indices/tesseract_data_frames/"
output_dir = "/home/jeusa/Files/Documents/Indexex/ILO_yearly_indices/extracted_indexes/"

if system()=="Windows":
    indexes_path = "H:\Documents\Indexex\ILO_yearly_indices\\tesseract_data_frames"
    output_dir = "H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes"

indexes_csv = util.list_files(indexes_path, suffix=".csv")

display(test_dfs, file)

['.././test_files/tesseract_data_frames/LS_index_1921.csv',
 '.././test_files/tesseract_data_frames/LS_index_1944.csv',
 '.././test_files/tesseract_data_frames/LS_index_1986_split.csv',
 '.././test_files/tesseract_data_frames/LS_index_1976.csv',
 '.././test_files/tesseract_data_frames/LS_index_1986.csv',
 '.././test_files/tesseract_data_frames/LS_index_1920.csv']

'.././test_files/tesseract_data_frames/LS_index_1976.csv'

In [3]:
pdf_file = test_files[4]
display(test_files, pdf_file)

['.././test_files/LS_index_1920.pdf',
 '.././test_files/LS_index_1937.pdf',
 '.././test_files/Argentinia_LS_index_1934_test_I_DE.pdf',
 '.././test_files/LS_index_1986.pdf',
 '.././test_files/LS_index_1976.pdf',
 '.././test_files/LS_index_1944_split.pdf',
 '.././test_files/LS_index_1944.pdf',
 '.././test_files/LS_index_1986_split.pdf',
 '.././test_files/LS_index_1921.pdf',
 '.././test_files/LS_index_1920_split.pdf']

'.././test_files/LS_index_1976.pdf'

In [17]:
# fitz mode
start_page = 1

pdf_words = util.read_pdf(pdf_file, start_page)
words_df = make_words_df(pdf_words, start_page)

r_df = extract_indexes(words_df, pdf_file, "fitz")

Reading pdf from .././test_files/LS_index_1976.pdf
...
Finished reading 30 page(s)


In [18]:
l_df = make_lines_df(words_df)
b_x0, b_x1, c0_c = group.group_line_starts_ends(l_df, "fitz")
bor = lines.make_borders_df(b_x0, b_x1)

In [20]:
# tess mode
pdf_df = pd.read_csv(file)
lines_df = lines.make_lines_df_from_ocr(pdf_df)
bins_x0, bins_x1, x0_n = group.group_line_starts_ends(lines_df, "tess")
borders = lines.make_borders_df(bins_x0, bins_x1)

lines_df

,line_text,artifact_text,x0,y0,x1,y1,page,dx
0,Hours of work: Arg. 1 (196-217); Bah. 1 (78-80...,,320,278,3542,352,1,3222
1,Night work: Arg. 1 (200). Trade unions: Eg. 1;...,,385,362,4449,427,1,4064
2,"Overtime: Arg. 1 (201, 203); Bah. 1.(79). (3-1...",,386,416,3770,479,1,3384
3,Shift work: Arg. 1 (202).,,388,481,857,524,1,469
4,Weekly rest: Arg. 1 (204-207); Bah. 1 (80); Ke...,,388,528,2970,585,1,2582
...,...,...,...,...,...,...,...,...
1770,United Kingdom (UK),,370,2499,785,2541,30,415
1771,1. (Great Britain) Trade unions and labour rel...,,439,2553,1736,2600,30,1297
1772,2. (Great Britain) Race relations,,435,2603,1068,2648,30,633
1773,International (Int.),,372,2700,718,2741,30,346


In [8]:
def make_words_df(words_list, start_page=1):

    page, words, x0, y0, x1, y1 = [], [], [], [], [], []

    for i, p in enumerate(words_list):
        for w in p:
            page.append(i+start_page)
            words.append(w[4])
            x0.append(w[0])
            y0.append(w[1])
            x1.append(w[2])
            y1.append(w[3])

    words_df = pd.DataFrame({
        "text": words,
        "x0": x0,
        "y0": y0,
        "x1": x1,
        "y1": y1,
        "page": page
    })

    return words_df


def make_words_df_from_ocr(pdf_df):
    
    df = pdf_df.copy().rename(columns={"left": "x0", "top": "y0", "page_num": "page"})
    df["x1"] = df["x0"] + df["width"]
    df["y1"] = df["y0"] + df["height"]
    
    df = df[["text", "x0", "y0", "x1", "y1", "page"]]
    
    return df

def make_lines_df(words_df):
    
    df = words_df.copy().rename(columns={"text": "line_text"})
    df = lines.merge_close_lines(df)
    
    return df

In [9]:
def extract_indexes_pdf(pdf_path, start_page=1, verbose=True, double_paged=None, save_to=None):

    pdf_words = util.read_pdf(pdf_path, start_page)
    words_df = make_words_df(pdf_words)
    
    ind_df = extract_indexes(words_df, file_name=os.path.basename(pdf_path), mode="fitz", verbose=verbose, double_paged=double_paged, save_to=save_to)

    return ind_df


def extract_indexes_tess(tess_df_path, start_page=1, verbose=True, double_paged=None, save_to=None):

    pdf_df = pd.read_csv(tess_df_path)
    pdf_df = pdf_df.loc[pdf_df["page_num"] >= start_page]
    ind_df = extract_indexes(pdf_df, file_name=os.path.basename(tess_df_path), mode="tess", verbose=verbose, double_paged=double_paged, save_to=save_to)

    return ind_df

def extract_indexes(df, file_name, mode, verbose=True, double_paged=None, save_to=None):
    lines_df = None
    words_df = df.copy()
    
    if mode=="fitz":
        lines_df = make_lines_df(words_df)
    elif mode=="tess":
        lines_df = lines.make_lines_df_from_ocr(df)
    
    bins_x0, bins_x1, x0_n = group.group_line_starts_ends(lines_df, mode)
    borders = lines.make_borders_df(bins_x0, bins_x1)

    if double_paged:
        return extract_double_paged_indexes(words_df, borders, file_name, mode, verbose)

    elif double_paged == None:
        if is_double_paged(words_df, borders, mode):
            return extract_double_paged_indexes(words_df, borders, file_name, mode, verbose)

    df = label.assign_types(lines_df, bins_x0, bins_x1, x0_n)
    df = label.assign_labels(df, x0_n)

    ind_df, p_l, p_g = label.correct_x0_types(df, bins_x0, bins_x1, x0_n, mode)
    ind_df = label.assign_labels(ind_df, x0_n)
    ind_df = label.approve_correction(df, ind_df, p_l)

    ind_df["new_label"] = ind_df["label"]
    ind_df = label.improve_country_classification(ind_df)

    ind_df = records.extract_records(ind_df)
    ind_df = extract.date.extract_dates(ind_df, file_name)
    ind_df = extract.clean_text(ind_df)

    if not save_to == None:
        ind_df.to_csv(save_to, index=False)

        if verbose:
            print(f"Saved extracted indexes to {save_to}.")

    return ind_df

In [10]:
def extract_double_paged_indexes(words_df, borders, file_name, mode, verbose=True):

    if verbose:
        print("Extracting indexes from document with double-pages.")

    df = words_df.copy()
    pdf_l = pd.DataFrame()
    pdf_r = pd.DataFrame()
    
    if mode=="tess":
        df = df.rename(columns={"left": "x0", "top": "y0", "page_num": "page"})

    for p, b in borders.groupby("page"):
        middle = b.iloc[0]["x0"] + b.iloc[0]["dx"]/2
        pdf_p = df.loc[df["page"] == p]

        l = pdf_p.loc[pdf_p["x0"] <= middle]
        r = pdf_p.loc[pdf_p["x0"] > middle]

        pdf_l = pd.concat([pdf_l, l])
        pdf_r = pd.concat([pdf_r, r])


    ind_l = extract_indexes(pdf_l, file_name, mode, verbose=verbose, double_paged=False)
    ind_r = extract_indexes(pdf_r, file_name, mode, verbose=verbose, double_paged=False)

    idx_s = ind_l.shape[0]
    idx_e = idx_s + ind_r.shape[0]
    ind_r = ind_r.set_index(pd.Index(list(range(idx_s, idx_e))))

    ind_df = pd.concat([ind_l, ind_r])
    ind_df = ind_df.rename_axis("idx").sort_values(by=["page", "idx"])
    ind_df = ind_df.reset_index(drop=True)

    return ind_df


def is_double_paged(words_df, borders, mode):
    df = words_df.copy()
    
    m = 0
    if mode=="fitz":
        m = 20
    elif mode=="tess":
        m = 100
        df = df.rename(columns={"left": "x0", "top": "y0", "page_num": "page"})
    else:
        raise ValueError(f"groups_lines() got an unknown value for parameter mode: {mode}") 
    
    double_p = []

    for p, b in borders.groupby("page"):
        middle = b.iloc[0]["x0"] + b.iloc[0]["dx"]/2

        middle_words = df.loc[(df["page"]==p) & (df["x0"] > middle-m) & (df["x0"] < middle+m)]
        if middle_words.shape[0] <= 1:
            double_p.append(True)
        else:
            double_p.append(False)

    if double_p.count(True) > 0.8 * len(double_p):
        return True

    return False

In [58]:
bor

,page,x0,x1,dx
0,1,60.088413,800.671143,740.582730
1,2,55.381128,804.133901,748.752773
2,3,50.879810,795.442122,744.562311
3,4,54.559184,386.214617,331.655434
4,5,45.599808,803.307711,757.707903
5,6,49.439610,803.609180,754.169570
6,7,53.999633,801.028271,747.028639
7,8,54.960188,802.208956,747.248768
8,9,49.199167,382.501208,333.302042
9,10,49.680187,804.806310,755.126123


In [57]:
p = 4
b = bor.loc[bor["page"]==p]
middle = b.iloc[0]["x0"] + b.iloc[0]["dx"]/2
df = words_df.copy()
m = 20

middle_words = df.loc[(df["page"]==p) & (df["x0"] > middle-m) & (df["x0"] < middle+m)]
display(middle, middle_words)

220.38690062775007

,text,x0,y0,x1,y1,page
2725,Insurance,225.356369,71.624084,258.880432,80.124084,4
2737,No.,206.159897,80.263489,219.106018,88.763489,4
2738,109.,223.440002,80.263489,237.783386,88.763489,4
2739,Health,239.519501,80.264282,262.261383,88.764282,4
2791,and,235.198898,105.944580,247.903519,114.444580,4
2803,11,230.638901,114.344971,238.014145,122.844971,4
2815,Amendment,215.760193,122.743958,255.117935,131.243958,4
2874,for,207.355087,140.984467,217.411911,149.484467,4
2875,the,219.352478,140.984467,229.890915,149.484467,4
2876,transfer,231.591354,140.984467,257.932190,149.484467,4


In [39]:
words_df

,text,x0,y0,x1,y1,page
0,Hours,58.319798,54.345398,78.633598,62.845398,1
1,of,82.077293,54.345398,88.397179,62.845398,1
2,work:,91.679413,54.345398,111.636101,62.845398,1
3,Arg.,114.479698,54.345398,129.822205,62.845398,1
4,1,133.681198,54.345398,137.931198,62.845398,1
...,...,...,...,...,...,...
26383,(Int.),113.280098,485.958984,130.480774,494.333984,30
26384,1.,79.919998,496.185852,86.076462,504.685852,30
26385,(Denmark-Finland-Iceland-Norway-Sweden),89.519897,496.092102,240.691406,504.717102,30
26386,Unemployment,251.760101,496.092102,305.602264,504.717102,30


In [19]:
is_double_paged(words_df, bor, "fitz") # not working

False

In [21]:
is_double_paged(pdf_df, borders, "tess")

True

In [11]:
extract_indexes_pdf(pdf_file, 1)

Reading pdf from .././test_files/LS_index_1976.pdf
...
Finished reading 30 page(s)


,country,region,text,date,year,page,extracted_date,full_text
0,ARGENTINA,,AUSTRALIA (cont.) AUSTRALIA (cont.) Queensland...,,,4,,AUSTRALIA (cont.) AUSTRALIA (cont.) Queensland...
1,AUSTRALIA,,and Accident Insurance (Amendment) Act (No. 6)...,,,5,,and Accident Insurance (Amendment) Act (No. 6)...
2,AUSTRIA,,20 Feb,,,6,,20 Feb.
3,AUSTRIA,,27 Feb. 8 Mar,,,6,,27 Feb. 8 Mar.
4,AUSTRIA,,22 Mar,,,6,,22 Mar.
...,...,...,...,...,...,...,...,...
104,THAILAND,Dominica,18 June,,,27,,18 June
105,THAILAND,Dominica,22 July Amendments to 20 CFR Part 404--Federal...,,,27,,22 July Amendments to 20 CFR Part 404--Federal...
106,THAILAND,Dominica,24 Dec. Austria-United Kingdom 14 Jan. Agreeme...,,,28,,24 Dec. Austria-United Kingdom 14 Jan. Agreeme...
107,THAILAND,Dominica,24 Dec. VENEZUELA,,,28,,24 Dec. VENEZUELA


In [13]:
extract_indexes_tess(file, 1)

Extracting indexes from document with double-pages.


,country,region,text,date,year,page,extracted_date,full_text
0,ARGENTINA,,1976—Arg. | This translation reproduces the of...,,,1,,1976—Arg. | This translation reproduces the of...
1,FRANCE,,1976—Fr. 1 This translation reproduces the tex...,,,1,,1976—Fr. 1 This translation reproduces the tex...
2,MALAYSIA,,Act 1967 (originally reprinted in the Legislat...,,,2,,Act 1967 (originally reprinted in the Legislat...
3,MALAYSIA,,as subsequently amended,,,2,,as subsequently amended.
4,PORTUGAL,,of Legislative Decree No. 215-B/75 of 30 April...,,,2,,of Legislative Decree No. 215-B/75 of 30 April...
...,...,...,...,...,...,...,...,...
656,INTERNATIONAL,France—Tunisia,Rom.),,,30,,(Rom.)
657,INTERNATIONAL,France—Tunisia,Sen.) Code (amendments) relationships,,,30,,(Sen.) Code (amendments) relationships
658,INTERNATIONAL,France—Tunisia,Swe.) injury insurance of parents to leave,,,30,,(Swe.) injury insurance of parents to leave
659,INTERNATIONAL,France—Tunisia,Tun.) Code (amendments) Soviet Socialist Repub...,,,30,,(Tun.) Code (amendments) Soviet Socialist Repu...


In [29]:
extract_indexes(words_df, pdf_file, "fitz")

Extracting indexes from document with double-pages.


,country,region,text,date,year,page,extracted_date,full_text
0,MALAYSIA,,Act 1967 (originally reprinted in the Legislat...,,,2,,Act 1967 (originally reprinted in the Legislat...
1,MALAYSIA,,as subsequently amended,,,2,,as subsequently amended.
2,PORTUGAL,,of Legislative Decree No. 215-B/75 of 30 April...,,,2,,of Legislative Decree No. 215-B/75 of 30 April...
3,PORTUGAL,,as amended by Legislative Decrees Nos. 773176 ...,,,2,,as amended by Legislative Decrees Nos. 773176 ...
4,PORTUGAL,,of 7 Dec. 1976,,,2,,of 7 Dec. 1976.
...,...,...,...,...,...,...,...,...
390,INTERNATIONAL,AustriaSweden,Sweden 1. Co-determination 2. Employment 3. Ri...,,,30,,Sweden 1. Co-determination 2. Employment 3. Ri...
391,INTERNATIONAL,AustriaSweden,Tunisia 1. Labour,,,30,,Tunisia 1. Labour
392,INTERNATIONAL,AustriaSweden,Union of 1. Inspection 2. (Russian,,,30,,Union of 1. Inspection 2. (Russian
393,INTERNATIONAL,AustriaSweden,United 1. (Great 2. (Great,,,30,,United 1. (Great 2. (Great


In [14]:
extract_double_paged_indexes(words_df, bor, pdf_file, "fitz")

Extracting indexes from document with double-pages.


,country,region,text,date,year,page,extracted_date,full_text
0,MALAYSIA,,Act 1967 (originally reprinted in the Legislat...,,,2,,Act 1967 (originally reprinted in the Legislat...
1,MALAYSIA,,as subsequently amended,,,2,,as subsequently amended.
2,PORTUGAL,,of Legislative Decree No. 215-B/75 of 30 April...,,,2,,of Legislative Decree No. 215-B/75 of 30 April...
3,PORTUGAL,,as amended by Legislative Decrees Nos. 773176 ...,,,2,,as amended by Legislative Decrees Nos. 773176 ...
4,PORTUGAL,,of 7 Dec. 1976,,,2,,of 7 Dec. 1976.
...,...,...,...,...,...,...,...,...
390,INTERNATIONAL,AustriaSweden,Sweden 1. Co-determination 2. Employment 3. Ri...,,,30,,Sweden 1. Co-determination 2. Employment 3. Ri...
391,INTERNATIONAL,AustriaSweden,Tunisia 1. Labour,,,30,,Tunisia 1. Labour
392,INTERNATIONAL,AustriaSweden,Union of 1. Inspection 2. (Russian,,,30,,Union of 1. Inspection 2. (Russian
393,INTERNATIONAL,AustriaSweden,United 1. (Great 2. (Great,,,30,,United 1. (Great 2. (Great


In [13]:
extract_double_paged_indexes(pdf_df, borders, file, "tess")

Extracting indexes from document with double-pages.


,country,region,text,date,year,page,extracted_date,full_text
0,ARGENTINA,,1976—Arg. | This translation reproduces the of...,,,1,,1976—Arg. | This translation reproduces the of...
1,FRANCE,,1976—Fr. 1 This translation reproduces the tex...,,,1,,1976—Fr. 1 This translation reproduces the tex...
2,MALAYSIA,,Act 1967 (originally reprinted in the Legislat...,,,2,,Act 1967 (originally reprinted in the Legislat...
3,MALAYSIA,,as subsequently amended,,,2,,as subsequently amended.
4,PORTUGAL,,of Legislative Decree No. 215-B/75 of 30 April...,,,2,,of Legislative Decree No. 215-B/75 of 30 April...
...,...,...,...,...,...,...,...,...
656,INTERNATIONAL,France—Tunisia,Rom.),,,30,,(Rom.)
657,INTERNATIONAL,France—Tunisia,Sen.) Code (amendments) relationships,,,30,,(Sen.) Code (amendments) relationships
658,INTERNATIONAL,France—Tunisia,Swe.) injury insurance of parents to leave,,,30,,(Swe.) injury insurance of parents to leave
659,INTERNATIONAL,France—Tunisia,Tun.) Code (amendments) Soviet Socialist Repub...,,,30,,(Tun.) Code (amendments) Soviet Socialist Repu...


In [49]:
is_double_paged(words_df, bor, "fitz")

False

In [18]:
#rec_df = extract.extract_indexes(pdf_df, save_to=test_output_dir + os.path.basename(file))
rec_df = extract_indexes(pdf_df, file, "tess")
rec_df

Extracting indexes from document with double-pages.


,country,region,text,date,year,page,extracted_date,full_text
0,ARGENTINA,,1976—Arg. | This translation reproduces the of...,,,1,,1976—Arg. | This translation reproduces the of...
1,FRANCE,,1976—Fr. 1 This translation reproduces the tex...,,,1,,1976—Fr. 1 This translation reproduces the tex...
2,MALAYSIA,,Act 1967 (originally reprinted in the Legislat...,,,2,,Act 1967 (originally reprinted in the Legislat...
3,MALAYSIA,,as subsequently amended,,,2,,as subsequently amended.
4,PORTUGAL,,of Legislative Decree No. 215-B/75 of 30 April...,,,2,,of Legislative Decree No. 215-B/75 of 30 April...
...,...,...,...,...,...,...,...,...
656,INTERNATIONAL,France—Tunisia,Rom.),,,30,,(Rom.)
657,INTERNATIONAL,France—Tunisia,Sen.) Code (amendments) relationships,,,30,,(Sen.) Code (amendments) relationships
658,INTERNATIONAL,France—Tunisia,Swe.) injury insurance of parents to leave,,,30,,(Swe.) injury insurance of parents to leave
659,INTERNATIONAL,France—Tunisia,Tun.) Code (amendments) Soviet Socialist Repub...,,,30,,(Tun.) Code (amendments) Soviet Socialist Repu...


In [7]:
rec_df.loc[rec_df["page"]==11]

,country,region,text,date,year,page,extracted_date,extracted_day,extracted_month,extracted_year,full_text
219,SWEDEN,,Act respecting the limitation of working hours...,17.10.,1919,11,17th October 1919,17,October,1919,Act respecting the limitation of working hours...
220,SWEDEN,,Act respecting hours of work on Swedish vessel...,24.10.,1919,11,24th October 1919,24,October,1919,Act respecting hours of work on Swedish vessel...
221,SWEDEN,,Royal Order issuing detailed regulations conce...,24.10.,1919,11,24th October 1919,24,October,1919,Royal Order issuing detailed regulations conce...
222,SWEDEN,,Royal notification respecting certain exemptio...,21.11.,1919,11,21st November 1919,21,November,1919,Royal notification respecting certain exemptio...
223,SWEDEN,,"Royal Order to amend the Order of “, No. 17, r...",27.2.,1920,11,27th February 1920,27,February,1920,"Royal Order to amend the Order of “30th March,..."
224,SWEDEN,,Act respecting conciliation in trade disputes....,28.5.,1920,11,28th May 1920,28,May,1920,Act respecting conciliation in trade disputes....
225,SWEDEN,,Act respecting the Central Arbitration Board f...,28.5.,1920,11,28th May 1920,28,May,1920,Act respecting the Central Arbitration Board f...
226,SWEDEN,,Act respecting special arbitrators in trade di...,28.5.,1920,11,28th May 1920,28,May,1920,Act respecting special arbitrators in trade di...
227,SWEDEN,,Royal notification relating to benefit to ‘une...,22.10.,1920,11,22nd October 1920,22,October,1920,Royal notification relating to benefit to ‘une...
228,SWITZERLAND,,Resolution of the Federal Council respecting u...,29.10.,1919,11,29th October 1919,29,October,1919,Resolution of the Federal Council respecting u...


## Probleme
- Dokumente mit 2 Spalten FIXED
- Dokumente mit Länder-Überschrift über ganze Breite der Seite IMPROVED
- Länder- und Regionenspalte vereinen?
    - Regionenerkennung oft nicht richtig, weil Groß- und Kleinbuchstaben vom OCR nicht richtig erkannt werden
- Dokumente/Seiten mit vielen Artifakten
    - 1944
        - S. 8-9, schlechte Ländererkennung FIXED
            - S. 25 falsches Labeling FIXED
        - S. 28 Inhalt fehlt? TESSERACT PROBLEM
        - S. 10-11, schlechte Ländererkennung FIXED
- Date-Type-Erkennung fehlerhaft
    - 1920 FIXED
- 1920 Seite 11 Uruguay
    - unvollständige Datums-Erkennung FIXED
    

## Todo
- doppelseitige Extraktion funktioniert auch im fitz-Modus
- Kommandozeilenprogramm